In [1]:
import random
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/knapsack.csv')
df

,Item,Weight,Value
0,Laptop,3.0,1500
1,Headphones,1.0,300
2,Book,2.0,120
3,Jacket,2.0,250
4,Camera,1.0,800
5,Sunglasses,0.5,200
6,Water Bottle,1.5,40
7,Notebook,1.0,60
8,Smartphone,0.5,1000
9,Portable Charger,1.0,150


In [4]:
def create_individual(length):
    """Create a random individual."""
    return [np.random.randint(0, 2) for _ in range(length)]

In [50]:
def compute_fitness(individual, object_list):
    """Compute the fitness of an individual. More 1s means higher fitness."""
    total_weigh = 0
    total_value = 0

    for i in range(len(individual)):
        if individual[i] == 1:
            total_weigh += object_list[i][1]
            total_value += object_list[i][2]

    if total_weigh > 15:
        return 0

    return total_value

In [51]:
def select_parents(population, fitnesses):
    """Select two parents based on their fitness. Higher fitness -> higher chance to be selected."""
    total_fitness = sum(fitnesses)
    selection_probs = [fitness/total_fitness for fitness in fitnesses]
    return random.choices(population, k=2, weights=selection_probs)

In [49]:
def crossover(parent1, parent2):
    """Perform a simple one-point crossover. Return two children."""
    crossover_point = np.random.randint(1, len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

In [82]:
def crossover2(parent1, parent2):    
    """Perform a two-point crossover. Return two children."""
    crossover_points = sorted(np.random.choice(range(1, len(parent1)), size=2, replace=False))
    crossover_point1, crossover_point2 = crossover_points

    child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
    child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
    return child1, child2

In [87]:
def mutate(individual, mutation_rate):
    """Flip random bits based on mutation_rate."""
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

In [88]:
def mutate2(individual, mutation_rate):
    """Scramble random bits based on mutation_rate."""
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            # Generate a random index to swap with
            j = random.randint(0, len(individual) - 1)
            # Swap the bits at index i and j
            individual[i], individual[j] = individual[j], individual[i]
    return individual

In [92]:
def run_genetic_algorithm(seq_length, population_size, generations, mutation_rate, object_list):
    """
    Run the genetic algorithm.

    seq_length: length of the bit string
    population_size: number of individuals in the population
    generations: number of generations to run
    mutation_prob: probability of mutation
    keep_best: whether to keep the best individual from the previous generation

    Return the final population.
    """

    # Initialize population
    population = [create_individual(seq_length) for _ in range(population_size)]
    fitnesses = [compute_fitness(individual, object_list) for individual in population]
    #print(fitnesses)

    best_fitness_ever = 0
    best_indi_ever = []
    

    for generation in range(generations):
        # Create new generation through selection, crossover, and mutation

        # Initialize new population
        new_population = []

        for _ in range(population_size // 2):
            # Select two parents
            parent1, parent2 = select_parents(population, fitnesses)

            # Crossover parents
            child1, child2 = crossover(parent1, parent2)

            # Mutate children
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # Add children to new population
            new_population.extend([child1, child2])

        # Update population and compute new fitnesses
        population = new_population
        fitnesses = [compute_fitness(individual, object_list) for individual in population]

        # # Print out the best fitness in this generation
        best_fitness = max(fitnesses)
        best_fitness_index = fitnesses.index(best_fitness)
        indi =  population[best_fitness_index]
        if best_fitness > best_fitness_ever:
            best_fitness_ever = best_fitness
            best_indi_ever = indi
           

        print(f"Generation {generation}, Best Fitness: {best_fitness}, Best Fitness Ever: {best_fitness_ever}, Best Individual Ever: {best_indi_ever} ")
        # print(f"Generation {generation}, Best Value: {best_value}, Best Weigh: {best_weigh}, Best Individual: {individual},")

    return population

In [78]:
object_list = df.values.tolist()
len(object_list)

30

In [93]:
# Run the GA
final_population = run_genetic_algorithm(seq_length=30, population_size=2000, generations=100, mutation_rate=0.1, object_list = object_list)

Generation 0, Best Fitness: 5720, Best Fitness Ever: 5720, Best Individual Ever: [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0] 
Generation 1, Best Fitness: 5730, Best Fitness Ever: 5730, Best Individual Ever: [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0] 
Generation 2, Best Fitness: 5700, Best Fitness Ever: 5730, Best Individual Ever: [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0] 
Generation 3, Best Fitness: 5880, Best Fitness Ever: 5880, Best Individual Ever: [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1] 
Generation 4, Best Fitness: 6200, Best Fitness Ever: 6200, Best Individual Ever: [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1] 
Generation 5, Best Fitness: 6040, Best Fitness Ever: 6200, Best Individual Ever: [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,